## Imports

In [156]:
from functools import lru_cache
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)
from time import sleep

## Request

Some structures used during requests are found below.

In [157]:
# Relevant data is found in two different URLs
urls = {
    'api': 'https://api.mtga.untapped.gg/api/v1/',
    'json': 'https://mtgajson.untapped.gg/v1/latest/',
}

# We read four JSONs in total. The following dictionary facilitates access to its endpoints.
# json: (url, endpoint) 
endpoints = {
    'active': ('api', 'meta-periods/active'),
    'analytics': ('api', 'analytics/query/card_stats_by_archetype_event_and_scope_free/ALL?MetaPeriodId='),
    'cards': ('json', 'cards.json'),
    'text': ('json', 'loc_en.json'),
}

# Header information
headers = {
    'authority': 'api.mtga.untapped.gg',
    'accept': '*/*',
    'accept-language': 'en-US,en;q=0.9,pt;q=0.8',
    'if-none-match': '"047066ff947f01e9e609ca4cf0d6c0a6"',
    'origin': 'https://mtga.untapped.gg',
    'referer': 'https://mtga.untapped.gg/',
    'sec-ch-ua': '"Google Chrome";v="111", "Not(A:Brand";v="8", "Chromium";v="111"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36',
}

In [158]:
@lru_cache
def request(keyword, send_headers=True, format=''):
    'Returns JSON from the corresponding keyword'
    url_kw, endpoint = endpoints[keyword]
    url = urls[url_kw]

    sleep(2) # Resonable interval betwween requests

    try:
        if send_headers:
            response = requests.get(url+endpoint+format, headers)
        else:
            response = requests.get(url+endpoint+format)

    except requests.exceptions.RequestException as error:
        raise error
    
    else:
        return response.json()

### Active JSON

We need to find what's the current active standard format to compose the analytics request.
We also get all the legal set codes contained in this format.

In [159]:
def request_active():
    'Returns format ID and lists of standard legal sets'

    # Extracting information from the latest standard BO1 format
    for format in reversed(request('active')):
        if format['event_name'] == 'Ladder':
            return str(format['id']), format['legal_sets']

format_id, legal_sets = request_active()
print(format_id, legal_sets)

355 ['MID', 'VOW', 'NEO', 'SNC', 'DMU', 'BRO', 'ONE', 'MOM']


### Cards JSON

We only bother to keep card information from the legal sets.

A card can be reprinted, i.e. a card from an older set could be reprinted in newer sets.
Untapped chooses to group all cards with the same title indicated by `grpid`, but for the sake of simplicity, we choose to only consider the latest reprint.

`raw_card` is a data frame containing card information with minimal modifications.

In [160]:
def request_cards(sets):
    'Returns raw card data frame'
    df = pd.DataFrame(request('cards'))

    # Ony bother with standard legal cards
    df = df[df.set.isin(sets)]

    # Remove duplicates by considering only the latest reprint
    gb = df.groupby('titleId').agg({'grpid':'max'})
    df = df[df.grpid.isin(gb.grpid)]

    return df.set_index('grpid')

raw_card = request_cards(legal_sets)

### Analytics JSON

`raw_analytics` contains daily games information separated by tiers.

In [161]:
def request_analytics(format):
    'Returns raw analytics data frame'
    json = request('analytics', False, format)

    return pd.json_normalize(json['data']).T

raw_analytics = request_analytics(format_id)

### Text JSON

All text used to build the web table is contained here, including card text.

`raw_text` is a data frame containing all text with minimal modifications.

In [162]:
def request_text():
    'Returns raw card text data frame'
    df = pd.DataFrame(request('text')).set_index('id')

    # Collapse columns raw and text, prioritizing raw
    df.loc[~df.raw.isna(), 'text'] = df.raw
    df.drop('raw', axis='columns', inplace=True)

    return df

raw_text = request_text()

## Get functions

After requesting all JSONs, we have to filter and normalize the data, which in this case results in multiple dataframes.

### Card information

The main function in the following block is `get_card_information()`. It gets data from `raw_card` and combines with relevant text from `text`. Normalization results in the following data frames with respective schema:

- `card`:
    - `card_id` (Integer PK): corresponding to `titleId` in JSON
    - `art_id` (Integer): art id in case I find a way to get images
    - `set_id` (String): three-letter set identifier
    - `title` (String): name of the card, necessarily unique
    - `rarity` (String): rarity of a card
    - `power` (String): creature power. Can be an integer, null or '*'
    - `toughness` (String): creature toughness. Can be an integer, null or '*'
    - `flavor` (String): flavor text, if a card has any
    - `is_legendary` (Boolean): flags a card is legendary
    - `is_token` (Boolean): flags a card token
    - `is_secondary_card` (Boolean): Flags a card secondary in an archetype
    - `is_rebalanced` (Boolean): Flags a digitally rebalanced card
- `card_type`:
    - `card_id` (Integer FK): corresponding to `titleId` in JSON
    - `type` (String PK): type according to magic rules
- `card_subtype`:
    - `card_id` (Integer FK): corresponding to `titleId` in JSON
    - `subtype` (String PK): subtype according to magic rules
- `card_cost`:
    - `card_id` (Integer FK): corresponding to `titleId` in JSON
    - `color` (String PK): six color identifiers according to magic rules
    - `cost` (Integer): amount of mana to be paid of given color
- `card_ability`
    - `card_id` (Integer FK): corresponding to `titleId` in JSON
    - `ability` (String): ability raw text

In [163]:
def filter_raw_card(raw_card):
    "Change naming conventions and keep only relevant columns before normalization"

    keep = [
        'titleId',
        'art_id',
        'flavorId',
        'power',
        'toughness',
        'set_id',
        'castingcost',
        'rarity',
        'cardTypeTextId',
        'subtypeTextId',
        'ability',
        'is_secondary_card',
        'is_token',
        'is_rebalanced'
    ]

    rename = {
        'set':'set_id',
        'isSecondaryCard': 'is_secondary_card',
        'isToken': 'is_token',
        'IsRebalanced': 'is_rebalanced',
        'artId': 'art_id',
        'abilities': 'ability'
    }

    return raw_card.rename(rename, axis = 'columns').reset_index()[keep]

In [164]:
def get_card_dataframe(df):
    "Returns card dataframe"   

    # Card columns mappable to text
    id_to_text = ['titleId', 'flavorId', 'cardTypeTextId', 'subtypeTextId']

    # Additional text_columns columns
    df = df.join(
        df[id_to_text]
        .applymap(lambda x: raw_text.loc[x].values[0], na_action='ignore')
        .rename(columns={column: column[:-2] for column in id_to_text}) # Remove Id
    )

    # Only tracked supertype will be 'legendary'
    df['is_legendary'] = df.cardTypeText.str.contains('Legendary')

    # Replace empty string flavor with NaN
    df.flavor = df.flavor.replace({'': np.nan})

    # Replace rarity id with text
    df.rarity = df.rarity.replace ({1: 'Common',
                                    2: 'Common',
                                    3: 'Uncommon',
                                    4: 'Rare',
                                    5: 'Mythic Rare'})

    # Define the columns and order for card data frame
    order = [
        'titleId',
        'art_id',
        'set_id',
        'title',
        'rarity',
        'power',
        'toughness',
        'flavor',
        'is_legendary',
        'is_token',
        'is_secondary_card',
        'is_rebalanced',
    ]

    return df[order]

In [165]:
def get_card_type(df):
    'Returns card type data frame'
    # Convert 'cardTypeTextId' to text
    df = df.join(df.cardTypeTextId
                   .map(lambda x: raw_text.loc[x].values[0], na_action='ignore')
                   .rename('type'))
    
    # Split the text and transform it into a list of rows, deleting the ones without information
    df = df.type.str.split().explode().dropna()

    # There are special cases of cards not having types such as cards 'Day' and 'Night'
    df = df[~df.isin(['NONE', 'Legendary', 'Basic', 'Token'])]

    return df

In [166]:
def get_card_subtype(df):
    "Returns card subtype data frame"
    # Convert 'subtypeTextId' to text
    df = df.join(df.subtypeTextId
                   .map(lambda x: raw_text.loc[x].values[0], na_action='ignore')
                   .rename('subtype'))

    # Split the text and transform it into a list of rows, deleting the ones without information
    df = df.subtype.str.split().explode().dropna()
    
    return df

In [167]:
def get_card_cost(df):
    "Returns card cost data frame"
    # Casting color codes in 'castingcost' column
    casting_colors = {
        'Black': r'oB',
        'Blue': r'oU',
        'Green': r'oG',
        'Red': r'oR',
        'White': r'oW',
        'Multicolor': r'\(',
        'X': r'oX'
    }

    # Create a column for each variety
    for color, code in casting_colors.items():
        df[color] = df.castingcost.str.count(code)

    # Special case is colorless that can be any number
    df['Colorless'] = df.castingcost.str.extract(r'(\d+)')

    # Stack columns into rows and set index to card_id only
    df = (df[['Colorless'] + list(casting_colors.keys())].stack()
                                                         .reset_index()
                                                         .set_index('card_id')
                                                         .rename(columns={'level_1': 'color', 0: 'cost'}))

    df.cost = pd.to_numeric(df.cost) # Convert from string to number

    # Only record costs above zero
    return df[df.cost > 0]

In [168]:
def get_card_ability(df):
    'Returns card_ability data frame'
    df = (df.ability
            .dropna()
            .explode()
            .apply(lambda x: x.get('TextId'))
            .map(lambda x: raw_text.loc[x].values[0]))      
    return df

In [169]:
def get_card_information():
    'Returns multiple data frames containing card information after normalization'
    filtered = filter_raw_card(raw_card)
    card = get_card_dataframe(filtered)

    # Rename 'titleId' to 'card_id' and promote it to index
    for df in [filtered, card]:
        df.set_index('titleId', inplace=True)
        df.index.name = 'card_id'

    card_type = get_card_type(filtered)
    card_subtype = get_card_subtype(filtered)
    card_cost = get_card_cost(filtered)
    card_ability = get_card_ability(filtered)

    return card, card_type, card_subtype, card_cost, card_ability

card, card_type, card_subtype, card_cost, card_ability = get_card_information()

### Analytics

The main function in the following block is `get_analytics()`. It gets data from `raw_analytics`. Normalization results in the following data frames with respective schema:

- `analytics_games`: Defined by `card_id`, `tier` and `copies`
    - `card_id` (Integer FK): corresponding to `titleId` in JSON
    - `tier` (String PK): game tier, from bronze to platinum
    - `copies` (Integer PK): number of copies, between 1 and 4
    - `games`(Integer): number of games
- `analytics_wins`: Defined by `card_id` and `tier`
    - `card_id` (Integer FK): corresponding to `titleId` in JSON
    - `tier` (String PK): game tier, from bronze to platinum
    - `wins` (Integer): number of games resulting in victory

In [170]:
def filter_raw_analytics():
    # Reset index and rename tiers
    df = (raw_analytics.reset_index()
                       .rename(columns={'index':'raw'}))
    
    # Split raw column into multiple
    df[['card_id', 'archetype_id', 'tier']] = df.raw.str.split('.', expand=True)

    # Record only consolidated data by archetype and change index to titleId
    df = (df[df.archetype_id == 'ALL']
            .drop(['raw', 'archetype_id'], axis=1)
            .set_index('card_id'))
    
    # Replace abbreviations with tier full names
    df.replace({'b': 'Bronze',
                's': 'Silver',
                'g': 'Gold',
                'p': 'Platinum'},
               inplace = True)

    # Unnest statistics
    unnest = ['games', 'wins', 'check', 'copies']
    df[unnest] = pd.DataFrame(df.explode([0])[0].to_list(), index=df.index).iloc[:, :4]

    return df[['tier', 'wins', 'copies']]

In [171]:
def get_card_tiered_daily_games(df):
    "Returns data frame 'analytics_games'"
    # Unnest copies
    unnest = [1, 2, 3, 4]
    df[unnest] = pd.DataFrame(df.copies.to_list(), index=df.index)

    df = (df.reset_index()
            .melt(id_vars = ['card_id', 'tier'],
                  value_vars = unnest,
                  var_name = 'copies',
                  value_name = 'games')
            .set_index('card_id')
            .dropna())
    
    return df

In [172]:
def get_analytics():
    "Returns both 'analytics_games' and 'analytics_wins'"
    filtered = filter_raw_analytics()

    analytics_wins = filtered[['tier', 'wins']]
    analytics_games = get_card_tiered_daily_games(filtered)
    
    return analytics_wins, analytics_games

analytics_wins, analytics_games = get_analytics()